# How to Block with MinHash LSH

Let's look at movies from IMBD.
Let's try to find all "related" movies, eg the sequels, prequels, and remakes.

They should have similar names, but they won't match exactly.
If we treat each title as a set of terms, eg the title `Batman Begins`
is `{"BATMAN", "BEGINS"}`, then we want to find other records that
have a high [jaccard similarity](https://en.wikipedia.org/wiki/Jaccard_index)
to this record.
This is what the MinHashLSH algorithm does:
allows us to find record pairs with high jaccard similarity in O(N) time.
This speedup isn't free though. The downside that we have to accept is
that MinHashLSH is probabilistic. Pairs with higher jacccard similarity
are more likely to be blocked together, but are not guaranteed.

Note that the data in this example isn't the ideal for MinHashLSH.
It would be better to use longer-form text, like a synopsis or description,
because then the size of our sets would be larger and more meaningful.

In [6]:
from __future__ import annotations

import ibis
from ibis import _

import mismo

ibis.options.interactive = True

In [7]:
t = ibis.examples.imdb_title_basics.fetch()
print(t.count())
t = t.head(1_000_000)
t

10814709


┏━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ tconst    ┃ titleType ┃ primaryTitle                                ┃ originalTitle                               ┃ isAdult ┃ startYear ┃ endYear ┃ runtimeMinutes ┃ genres                   ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string    │ string    │ string                                      │ string                                      │ int64   │ int64     │ string  │ int64          │ string                   │
├───────────┼───────────┼─────────────────────────────────────────────┼─────────────────────────────────────────────┼─────────┼───────────┼─────────┼────────────────┼──────────────────────────┤
│ tt0000001 │ short     │ Carmencita                                  │ Carmencita                                  │       0 │      1894 │ NULL    │              1 │ Documentary,Short        │
│ tt0000002 │ short     │ Le clown et ses chiens                      │ Le clown et ses chiens                      │       0 │      1892 │ NULL    │              5 │ Animation,Short          │
│ tt0000003 │ short     │ Pauvre Pierrot                              │ Pauvre Pierrot                              │       0 │      1892 │ NULL    │              4 │ Animation,Comedy,Romance │
│ tt0000004 │ short     │ Un bon bock                                 │ Un bon bock                                 │       0 │      1892 │ NULL    │             12 │ Animation,Short          │
│ tt0000005 │ short     │ Blacksmith Scene                            │ Blacksmith Scene                            │       0 │      1893 │ NULL    │              1 │ Comedy,Short             │
│ tt0000006 │ short     │ Chinese Opium Den                           │ Chinese Opium Den                           │       0 │      1894 │ NULL    │              1 │ Short                    │
│ tt0000007 │ short     │ Corbett and Courtney Before the Kinetograph │ Corbett and Courtney Before the Kinetograph │       0 │      1894 │ NULL    │              1 │ Short,Sport              │
│ tt0000008 │ short     │ Edison Kinetoscopic Record of a Sneeze      │ Edison Kinetoscopic Record of a Sneeze      │       0 │      1894 │ NULL    │              1 │ Documentary,Short        │
│ tt0000009 │ movie     │ Miss Jerry                                  │ Miss Jerry                                  │       0 │      1894 │ NULL    │             45 │ Romance                  │
│ tt0000010 │ short     │ Leaving the Factory                         │ La sortie de l'usine Lumière à Lyon         │       0 │      1895 │ NULL    │              1 │ Documentary,Short        │
│ …         │ …         │ …                                           │ …                                           │       … │         … │ …       │              … │ …                        │
└───────────┴───────────┴─────────────────────────────────────────────┴─────────────────────────────────────────────┴─────────┴───────────┴─────────┴────────────────┴──────────────────────────┘

We create our terms from the tokens in the `primaryTitle` column.
As an additional step, we drop common stopwords such as "The".

In [3]:
t = t.select(
    record_id=ibis.row_number(),
    primaryTitle=_.primaryTitle,
    terms=_.primaryTitle.upper().split(" "),
)
t = mismo.arrays.array_filter_isin_other(
    t, "terms", mismo.sets.rare_terms(t.terms, max_records_frac=0.05)
)
t = t.cache()
t

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id ┃ primaryTitle                                ┃ terms                              ┃ terms_filtered                         ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64     │ string                                      │ array<string>                      │ array<string>                          │
├───────────┼─────────────────────────────────────────────┼────────────────────────────────────┼────────────────────────────────────────┤
│         0 │ Carmencita                                  │ ['CARMENCITA']                     │ ['CARMENCITA']                         │
│         1 │ Le clown et ses chiens                      │ ['LE', 'CLOWN', ... +3]            │ ['LE', 'CLOWN', ... +3]                │
│         2 │ Pauvre Pierrot                              │ ['PAUVRE', 'PIERROT']              │ ['PAUVRE', 'PIERROT']                  │
│         3 │ Un bon bock                                 │ ['UN', 'BON', ... +1]              │ ['UN', 'BON', ... +1]                  │
│         6 │ Corbett and Courtney Before the Kinetograph │ ['CORBETT', 'AND', ... +4]         │ ['CORBETT', 'AND', ... +3]             │
│         7 │ Edison Kinetoscopic Record of a Sneeze      │ ['EDISON', 'KINETOSCOPIC', ... +4] │ ['EDISON', 'KINETOSCOPIC', ... +3]     │
│         8 │ Miss Jerry                                  │ ['MISS', 'JERRY']                  │ ['MISS', 'JERRY']                      │
│        10 │ Akrobatisches Potpourri                     │ ['AKROBATISCHES', 'POTPOURRI']     │ ['AKROBATISCHES', 'POTPOURRI']         │
│        12 │ The Photographical Congress Arrives in Lyon │ ['THE', 'PHOTOGRAPHICAL', ... +4]  │ ['PHOTOGRAPHICAL', 'CONGRESS', ... +3] │
│        13 │ The Waterer Watered                         │ ['THE', 'WATERER', ... +1]         │ ['WATERER', 'WATERED']                 │
│         … │ …                                           │ …                                  │ …                                      │
└───────────┴─────────────────────────────────────────────┴────────────────────────────────────┴────────────────────────────────────────┘

In MinHashLSH, the probability that a pair is blocked together is a function
of their jaccard similarity. We can tune this probability using the
`band_size` and `n_bands` params of `MinhashLshBlocker`:

In [4]:
mismo.block.plot_lsh_curves()

alt.Chart(...)

In [5]:
# Performance is still very bad. For now filter to a smaller dataset.
# Hopefully we can keep the API something like this,
# but it may have to change to get the performance we need.
k = 5_000
batmans = t.filter(_.primaryTitle.upper().contains("BATMAN"))
to_block = ibis.union(batmans, t.head(k))

# The time required is related to the band size and number of bands,
# so keep them small as small as possible.
blocker = mismo.block.MinhashLshBlocker(
    terms_column="terms_filtered", band_size=5, n_bands=10
)
blocked = blocker(to_block, to_block)
blocked2 = blocked.filter(_.primaryTitle_l.upper() != _.primaryTitle_r.upper())
blocked2 = blocked2.cache()
blocked2

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id_l ┃ record_id_r ┃ primaryTitle_l                                ┃ primaryTitle_r                                  ┃ terms_filtered_l            ┃ terms_filtered_lsh_keys_l                            ┃ terms_filtered_lsh_keys_r                            ┃ terms_filtered_r                   ┃ terms_l                     ┃ terms_r                           ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64       │ int64       │ string                                        │ string                                          │ array<string>               │ array<uint64>                                        │ array<uint64>                                        │ array<string>                      │ array<string>               │ array<string>                     │
├─────────────┼─────────────┼───────────────────────────────────────────────┼─────────────────────────────────────────────────┼─────────────────────────────┼──────────────────────────────────────────────────────┼──────────────────────────────────────────────────────┼────────────────────────────────────┼─────────────────────────────┼───────────────────────────────────┤
│        3553 │      485811 │ With General Pancho Villa in Mexico           │ I Hate Living with You, Batman                  │ ['WITH', 'GENERAL', ... +4] │ [3979803048262214771, 3905098727700485383, ... +3]   │ [2664953692877364926, 2690838631649474936, ... +3]   │ ['I', 'HATE', ... +4]              │ ['WITH', 'GENERAL', ... +4] │ ['I', 'HATE', ... +4]             │
│        4623 │      511954 │ Sport and Travel in Central Africa            │ I've Got Batman in My Basement                  │ ['SPORT', 'AND', ... +4]    │ [14400883293101828366, 14925453761580254457, ... +3] │ [13975641331648589312, 3724507162256288720, ... +3]  │ ["I'VE", 'GOT', ... +4]            │ ['SPORT', 'AND', ... +4]    │ ["I'VE", 'GOT', ... +4]           │
│         101 │      511954 │ The Beach at Villiers in a Gale               │ I've Got Batman in My Basement                  │ ['BEACH', 'AT', ... +4]     │ [10231976821963998966, 2962866122194191642, ... +3]  │ [13975641331648589312, 3724507162256288720, ... +3]  │ ["I'VE", 'GOT', ... +4]            │ ['THE', 'BEACH', ... +5]    │ ["I'VE", 'GOT', ... +4]           │
│        1452 │      999928 │ When We Were in Our Teens                     │ Cowabunga, Batman! When Comic Worlds Collide    │ ['WHEN', 'WE', ... +4]      │ [6161775411908023008, 13049240162514851464, ... +2]  │ [16253383743560961366, 5644658368434604354, ... +3]  │ ['COWABUNGA,', 'BATMAN!', ... +4]  │ ['WHEN', 'WE', ... +4]      │ ['COWABUNGA,', 'BATMAN!', ... +4] │
│         123 │         190 │ Tempête sur la jetée du treport               │ Déménagement à la cloche de bois                │ ['TEMPÊTE', 'SUR', ... +4]  │ [13648411972125653730, 16593680511188241775, ... +3] │ [12209079947480586911, 12303716028845945890, ... +4] │ ['DÉMÉNAGEMENT', 'À', ... +4]      │ ['TEMPÊTE', 'SUR', ... +4]  │ ['DÉMÉNAGEMENT', 'À', ... +4]     │
│         101 │         210 │ The Beach at Villiers in a Gale               │ The Astronomer's Dream; or, the Man in the Moon │ ['BEACH', 'AT', ... +4]     │ [10231976821963998966, 2962866122194191642, ... +3]  │ [3484159417531669044, 4609451508134221726, ... +4]   │ ["ASTRONOMER'S", 'DREA